In [ ]:
import os
import requests
from datetime import datetime
from dotenv import load_dotenv
load_dotenv()

# read variables from the .env file
RADAR_URL = os.getenv('RADAR_URL')
RADAR_CLIENT_ID = os.getenv('RADAR_CLIENT_ID')
RADAR_CLIENT_SECRET = os.getenv('RADAR_CLIENT_SECRET')
RADAR_REDIRECT_URL = os.getenv('RADAR_REDIRECT_URL')
RADAR_USERNAME = os.getenv('RADAR_USERNAME')
RADAR_PASSWORD = os.getenv('RADAR_PASSWORD')
RADAR_CONTRACT_ID = os.getenv('RADAR_CONTRACT_ID')
RADAR_WORKSPACE_ID = os.getenv('RADAR_WORKSPACE_ID')
RADAR_EMAIL = os.getenv('RADAR_EMAIL')
RADAR_BACKLINK = os.getenv('RADAR_BACKLINK')

RADAR_API_URL = RADAR_URL + '/radar/api'
RADAR_INGEST_URL = RADAR_URL + '/radar-ingest'

In [ ]:
# obtain token
response = requests.post(RADAR_API_URL + '/tokens', json={
    'clientId': RADAR_CLIENT_ID,
    'clientSecret': RADAR_CLIENT_SECRET,
    'redirectUrl': RADAR_REDIRECT_URL,
    'userName': RADAR_USERNAME,
    'userPassword': RADAR_PASSWORD
})

try:
    response.raise_for_status()
except requests.exceptions.HTTPError as e: 
    print(e, r.text)

tokens = response.json()
headers = {
    'Authorization': 'Bearer {}'.format(tokens['access_token'])
}
tokens, headers

In [ ]:
 # list contracts
url = RADAR_API_URL + '/contracts'
response = requests.get(url, headers=headers)
response.json()

In [ ]:
# get contract
url = RADAR_API_URL + '/contracts'.format(RADAR_CONTRACT_ID)
response = requests.get(url, headers=headers)
response.json()

In [ ]:
# create workspace
url = RADAR_API_URL + '/contracts/{}/workspaces'.format(RADAR_CONTRACT_ID)
response = requests.post(url, headers=headers, json={
    "descriptiveMetadata":{
        "title": "Test workspace",
        "description": "This is the test workspace"
    },
    "technicalMetadata": {
        "defaultDatasetRetentionPeriod": 42,
        "defaultDatasetResponsibleEmail": "mail@example.com"
    }
})
response.json()

In [ ]:
# list workspaces
url = RADAR_API_URL + '/workspaces'
response = requests.get(url, headers=headers)
response.json()

In [ ]:
# create dataset
url = RADAR_API_URL + '/workspaces/{}/datasets'.format(RADAR_WORKSPACE_ID)
archive_date = int(datetime.utcnow().timestamp())
publish_date = int(datetime.strptime('2020-12-10', '%Y-%m-%d').timestamp())
response = requests.post(url, headers=headers, json={
    "technicalMetadata": { 
        "retentionPeriod": 10,
        "archiveDate": archive_date,
        "publishDate": publish_date,
        "responsibleEmail": 'admin@example.com',
        # "publicationBacklink": "http://alice-bob.de/publication",
        # "embargoEndingDate": 1453982839735
    },
    "descriptiveMetadata": {
        #"alternateIdentifiers": null,
        #"relatedIdentifiers": null,
        "creators": {
            "creator": [
                {
                    'creatorName': 'Max Mustermann',
                    'creatorAffiliation': 'Scientist'
                },
                {  
                    'creatorName': 'Maike Maler',
                    'creatorAffiliation': 'Scientist'
                }
            ]
        },
        #"contributors": CONTRIBUTORS,
        "title": "Test dataset",
        #"descriptions": null,
        #"keywords": null,
        "publishers": {
            'publisher': ['Karlsruher Institut für Technologie (KIT)']
        },
        "productionYear": '2020',
        "publicationYear": '2020',
        "language": 'ENG',
        #"subjectAreas":null,
        #"resource":null,
        #"geoLocations":null,
        #"dataSources":null,
        #"software":null,
        #"processing":null,
        #"rights":null,
        #"rightsHolders":null,
        #"relatedInformations":null
    }
})
dataset = response.json()
dataset

In [ ]:
# upload files
url = dataset.get('uploadUrl')
files = {'upload_file': open('README.md','rb')}
response = requests.post(url, files=files, headers=headers)
response.text